In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split # Mirar la libreria sklearn.model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report

from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn import preprocessing
from sklearn import linear_model

import warnings
warnings.simplefilter('ignore')

%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Modelo de evolución de un disco protoplanetario en un sistema planetario
### A continuación se importan los datos de la evolución de un disco de gas y polvo que rodea una estrella central y la evolución de este en un sistema planetario

In [2]:
high = pd.read_csv('datos/highp_finalresults.csv')
low = pd.read_csv('datos/lowp_finalresults.csv')

In [3]:
high.tail()

,ident,it,t,a(i),emegas(i),emepla(i)/emet,rplanet(i)/radtie,emestar,rc,qest,sigmag_0,emed,gama,apert,fpert,constmigI,emetal,taugas
32460,2974,199875,20000001.0,4.403352,0.0,0.568129,1.2753727397384054,4.092467e+32,50.698605,6.853647,9.907135,0.018,1.0,0.3,1.0,0.1,0.408212,7710610.5
32461,2974,199875,20000001.0,4.998909,0.0,0.346168,1.0811722783677506,4.092467e+32,50.698605,6.853647,9.907135,0.018,1.0,0.3,1.0,0.1,0.408212,7710610.5
32462,2974,199875,20000001.0,5.472611,0.0,0.205236,0.90869255913863689,4.092467e+32,50.698605,6.853647,9.907135,0.018,1.0,0.3,1.0,0.1,0.408212,7710610.5
32463,2974,199875,20000001.0,6.079242,0.0,0.285096,1.0137383929400392,4.092467e+32,50.698605,6.853647,9.907135,0.018,1.0,0.3,1.0,0.1,0.408212,7710610.5
32464,2974,199875,20000001.0,7.411129,0.0,0.074660,0.64875258486623388,4.092467e+32,50.698605,6.853647,9.907135,0.018,1.0,0.3,1.0,0.1,0.408212,7710610.5


In [4]:
low.head()

,ident,it,t,a(i),emegas(i),emepla(i)/emet,rplanet(i)/radtie,emestar,rc,qest,sigmag_0,emed,gama,apert,fpert,constmigI,emetal,taugas
0,0,199992,20000001.0,0.026231,0.001185,1.710156,1.8388847169925493,4.368174e+32,59.88237,1.110726,51.287674,0.13,1.0,0.1,1.0,0.1,0.07707,9268935.0
1,0,199992,20000001.0,2.170653,0.000000,1.646707,1.8167126017756039,4.368174e+32,59.88237,1.110726,51.287674,0.13,1.0,0.1,1.0,0.1,0.07707,9268935.0
2,0,199992,20000001.0,4.251030,0.000000,1.059631,1.5688831343393115,4.368174e+32,59.88237,1.110726,51.287674,0.13,1.0,0.1,1.0,0.1,0.07707,9268935.0
3,0,199992,20000001.0,5.468018,0.000000,0.839156,1.4520602505243447,4.368174e+32,59.88237,1.110726,51.287674,0.13,1.0,0.1,1.0,0.1,0.07707,9268935.0
4,0,199992,20000001.0,8.935596,0.000000,0.367578,1.1032029110256696,4.368174e+32,59.88237,1.110726,51.287674,0.13,1.0,0.1,1.0,0.1,0.07707,9268935.0


In [5]:
num_planetas_high = high.groupby(['ident']).size().get_values()
num_planetas_low = low.groupby(['ident']).size().get_values()

In [6]:
masa_plan_high = high.groupby(['ident'])['emepla(i)/emet'].sum().get_values()
masa_plan_low = low.groupby(['ident'])['emepla(i)/emet'].sum().get_values()

In [7]:
efic_masa_high = masa_plan_high/high.groupby(['ident'])['emed'].mean().get_values()
efic_masa_low = masa_plan_low/low.groupby(['ident'])['emed'].mean().get_values()

In [8]:
X_low = low.drop(['a(i)','emegas(i)','emepla(i)/emet','rplanet(i)/radtie'],inplace=False,axis=1).groupby(['ident']).mean()
X_high = high.drop(['a(i)','emegas(i)','emepla(i)/emet','rplanet(i)/radtie'],inplace=False,axis=1).groupby(['ident']).mean()

In [9]:
scaler = preprocessing.StandardScaler().fit(X_high)
scaler
Scale_high = scaler.transform(X_high)
scaler = preprocessing.StandardScaler().fit(X_low)
scaler
Scale_low = scaler.transform(X_low)

## Modelo de estimación del número de planetas

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(Scale_high, num_planetas_high, test_size=0.3)

In [11]:
C=0.9
gamma=0.1
gauss = svm.SVC(C=C,kernel='rbf',gamma=gamma)
gauss.fit(X_train,Y_train)
Y_predict = gauss.predict(X_test)
ii = Y_predict == Y_test
print(sum(ii))

274


In [12]:
C=0.6
gamma=0.1
linear = svm.SVC(C=C,kernel='linear')
linear.fit(X_train,Y_train)
Y_predict = linear.predict(X_test)
ii = Y_predict == Y_test
print(sum(ii))

279


In [13]:
C=1
gamma=0.1
poly = svm.SVC(C=C,kernel='poly',degree=3)
poly.fit(X_train,Y_train)
Y_predict = poly.predict(X_test)
ii = Y_predict == Y_test
print(sum(ii))

107


In [14]:
clf = linear_model.Lasso(alpha=0.001, tol=0.00001)
clf.fit(X_train, Y_train)
Y_predict = clf.predict(X_test)
ii = 0
for i in range(len(Y_predict)):
    ii+=round(Y_predict[i])==Y_test[i]
ii

239

In [15]:
clf2 = svm.SVC(C=C,kernel='linear')
scores = cross_val_score(clf2, Scale_high, num_planetas_high, cv=20)
scores

array([ 0.34      ,  0.37162162,  0.34246575,  0.37671233,  0.35172414,
        0.32867133,  0.38461538,  0.3028169 ,  0.3028169 ,  0.30985915,
        0.28571429,  0.31386861,  0.34328358,  0.36842105,  0.30827068,
        0.35877863,  0.35384615,  0.4       ,  0.34375   ,  0.38582677])

In [16]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

print("# Tuning hyper-parameters for none score")
print()

print(np.unique(Y_train))

clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=2)
clf.fit(Scale_high, num_planetas_high)

# Tuning hyper-parameters for none score

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 26]


GridSearchCV(cv=2, error_score='raise',
       estimator=SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'kernel': ['rbf'], 'gamma': [0.001, 0.0001], 'C': [1, 10, 100, 1000]}, {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [17]:
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

Best parameters set found on development set:

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.136 (+/-0.001) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}

0.127 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}

0.189 (+/-0.036) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

0.135 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}

0.338 (+/-0.002) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}

0.189 (+/-0.041) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}

0.342 (+/-0.005) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}

0.332 (+/-0.002) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}

0.335 (+/-0.002) for {'C': 1, 'kernel': 'linear'}

0.336 (+/-0.006) for {'C': 10, 'kernel': 'linear'}

0.335 (+/-0.011) for {'C': 100, 'kernel': 'linear'}

0.335 (+/-0.004) for {'C': 1000, 'kernel': 'linear'}



In [18]:
clf = GridSearchCV(SVC(C=1), tuned_parameters,
                       scoring='%s_macro' % scores[1])
clf.fit(X_train,Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'kernel': ['rbf'], 'gamma': [0.001, 0.0001], 'C': [1, 10, 100, 1000]}, {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall_macro', verbose=0)

###  Support Vector Machine con kernel gaussiano
Se exploran parámetros en una escala logarítmica para optimizar el ajuste.

In [19]:
# unos for's muy grandes para todos los posibles metodos
#RBF
C=[0.1, 0.01, 0.001, 0.0001, 0.00001, 1, 5]
gamma=[0.1, 0.01, 0.001, 0.0001, 0.00001, 1, 10, 100, 1000, 10000]
sco=0
for c in C:
    for g in gamma:
        gauss = svm.SVC(C=c,kernel='rbf',gamma=g)
        gauss.fit(X_train,Y_train)
        Y_predict = gauss.predict(X_test)
        ii = Y_predict == Y_test
        if sum(ii) > sco:
            sco = sum(ii)
            mejor_g = g
            mejor_c = c
print('metodo: SVM rbf', '. hits:',sco, '. mejor gamma:',mejor_g, '. mejor c:', mejor_c)

metodo: SVM rbf . hits: 279 . mejor gamma: 0.1 . mejor c: 5


###  Support Vector Machine con kernel lineal
Se exploran parámetros en una escala logarítmica para optimizar el ajuste.

In [20]:
#linear
C = [0.1, 0.01, 0.001, 0.0001, 0.00001, 1, 5]
sco = 0
for c in C:
    linear = svm.SVC(C=c,kernel='linear')
    linear.fit(X_train,Y_train)
    Y_predict = linear.predict(X_test)
    ii = Y_predict == Y_test
    if sum(ii) > sco:
        sco = sum(ii)
        mejor_c = c
print('metodo: SVM linear', '. hits:',sco, '. mejor c:', mejor_c)

metodo: SVM linear . hits: 277 . mejor c: 1


###  Support Vector Machine con kernel polinomial
Se exploran parámetros en una escala logarítmica para optimizar el ajuste.

In [21]:
#poly
C=[0.1, 0.01, 0.001, 0.0001, 0.00001, 1, 5]
degr = [1, 2, 3, 4]
sco = 0
for c in C:
    for d in degr:
        poly = svm.SVC(C=c,kernel='poly',degree=d)
        poly.fit(X_train,Y_train)
        Y_predict = poly.predict(X_test)
        ii = Y_predict == Y_test
        if sum(ii) > sco:
            sco = sum(ii)
            mejor_d = d
            mejor_c = c
print('metodo: SVM poly', '. hits:',sco, '. mejor grado:',mejor_d, '. mejor c:', mejor_c)

metodo: SVM poly . hits: 277 . mejor grado: 1 . mejor c: 5


###  Lasso
Se exploran parámetros en una escala logarítmica para optimizar el ajuste.

In [22]:
#Lasso
alph = [0.1, 0.01, 0.001, 0.0001, 0.00001, 1, 10, 100, 1000, 10000]
sco = 0
for a in alph:
    clf = linear_model.Lasso(alpha=a, tol=0.00001)
    clf.fit(X_train, Y_train)
    Y_predict = clf.predict(X_test)
    ii = 0
    for i in range(len(Y_predict)):
        ii+=round(Y_predict[i])==Y_test[i]
    if ii > sco:
        sco = ii
        mejor_a = a
print('metodo: Lasso', '. hits:',sco, '. mejor alpha:',mejor_a)

metodo: Lasso . hits: 241 . mejor alpha: 0.01


## Modelo de estimación de la masa total de los planetas

In [23]:
X_train_high, X_test_high, Y_train_high, Y_test_high = train_test_split(Scale_high, masa_plan_high, test_size=0.3)
X_train_low, X_test_low, Y_train_low, Y_test_low = train_test_split(Scale_low, masa_plan_low, test_size=0.3)

### Support vector regression con score de ($\frac{1}{n} \sum^{n}_{i=1} ((y_i - \hat{y}_i)/y_i)^2)$)

In [24]:
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3, degree=2)
Y_rbf = svr_rbf.fit(X_train_high, Y_train_high).predict(X_test_high)
err = (((Y_rbf - Y_test_high)/Y_test_high)**2).mean()
print ("Error de aproximación ", err)

Y_lin = svr_lin.fit(X_train_high, Y_train_high).predict(X_test_high)
err = (((Y_lin - Y_test_high)/Y_test_high)**2).mean()
print ("Error de aproximación ", err)

Y_poly = svr_poly.fit(X_train_high, Y_train_high).predict(X_test_high)
err = (((Y_poly - Y_test_high)/Y_test_high)**2).mean()
print ("Error de aproximación ", err)

Error de aproximación  6.85038811797
Error de aproximación  12.0180479211
Error de aproximación  7.36797118864


### Support Vector Regression con kernel gaussiano

In [25]:
# unos for's muy grandes para todos los posibles metodos
#RBF
C=[0.1, 0.01, 0.001, 0.0001, 0.00001, 1, 5]
gamma=[0.1, 0.01, 0.001, 0.0001, 0.00001, 1, 10, 100, 1000, 10000]
sco=0
for c in C:
    for g in gamma:
        svr_rbf = SVR(kernel='rbf', C=c, gamma=g)
        Y_rbf = svr_rbf.fit(X_train_high, Y_train_high).predict(X_test_high)
        err = (((Y_rbf - Y_test_high)/Y_test_high)**2).mean()
        if (1/err) > sco:
            sco = 1/err
            mejor_g = g
            mejor_c = c
print('metodo: SVR rbf', '. score:',(1/sco), '. mejor gamma:',mejor_g, '. mejor c:', mejor_c)

metodo: SVR rbf . score: 0.697855412533 . mejor gamma: 0.1 . mejor c: 1


### Support Vector Regression con kernel lineal

In [26]:
#linear
C = [0.1, 0.01, 0.001, 0.0001, 0.00001, 1, 5]
sco = 0
for c in C:
    svr_lin = SVR(kernel='linear', C=c)
    Y_lin = svr_lin.fit(X_train_high, Y_train_high).predict(X_test_high)
    err = (((Y_lin - Y_test_high)/Y_test_high)**2).mean()
    if (1/err) > sco:
        sco = 1/err
        mejor_g = g
        mejor_c = c
print('metodo: SVR linear', '. score:',(1/sco), '. mejor c:', mejor_c)

metodo: SVR linear . score: 6.07765296606 . mejor c: 0.01


### Support Vector Regression con kernel polinomial

In [27]:
#poly
C=[0.1, 0.01, 0.001, 0.0001, 0.00001, 1, 5]
degr = [1, 2, 3, 4]
sco = 0
for c in C:
    for d in degr:
        svr_poly = SVR(kernel='poly', C=c, degree=d)
        Y_poly = svr_poly.fit(X_train_high, Y_train_high).predict(X_test_high)
        err = (((Y_poly - Y_test_high)/Y_test_high)**2).mean()
        if (1/err) > sco:
            sco = 1/err
            mejor_g = g
            mejor_c = c
print('metodo: SVR poly', '. score:',(1/sco), '. mejor grado:',mejor_d, '. mejor c:', mejor_c)

metodo: SVR poly . score: 5.73725398382 . mejor grado: 1 . mejor c: 0.1
